# Duck Curve
Duck curve from Pecan Street data is already calculated and visualized, go check out how they did that.

## OhmConnect
Wholesale scaling/tuning of IoT and interconnected power sources to shift demand rather than shifting supply. EX: Demand can become supply if there is a car that can send power back to grid.

## Austin Energy
Would like to check if duck curve can be shifted by managing behavior.

### incentivizing using certain devices at specific times or when not to run
Dishwasher, fridge, etc.

### Automated control (like ohmconnect) would require the price point of wholesale energy needs to be lower than switching on new production. Also, ohm connect needs to produce profit, cost = incentivizing customers, paying x per kwh, revenue = selling that energy to wholesale.

## Could create simulation that represents devices being shut off, how much could this shift demand, either manually or automatically.

# Data
## Ithaca vs Austin
find similar homes in austin than there are in ithaca: home size, no. of occupants and has pv.

## Duck curve
First, get a random day, get demand curve for that houses in ithaca, then austin. Look at the difference in curves.

Second, get hourly avg demand for a given season (possibly find statistical distribution for a month/season and select days that are a representative sample of the months weather variation).

Compare monthly/seasonal data in plots, austin vs ithaca, line for each season which consists of averaged hourly demand.

In [1]:
import os
import numpy as np
import pandas as pd
import pyodbc
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
# read in metadata to get house id's
meta_df = pd.read_csv('metadata.csv')
meta_df.drop(0,inplace=True)
meta_df.head()

,dataid,active_record,building_type,city,state,egauge_1min_min_time,egauge_1min_max_time,egauge_1min_data_availability,egauge_1s_min_time,egauge_1s_max_time,...,survey_2017,survey_2019,program_579,program_baseline,program_energy_internet_demo,program_lg_appliance,program_verizon,program_ccet_group,program_civita_group,program_shines
1,2836,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN
2,2743,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN
3,5323,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN
4,8560,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN
5,3313,NaN,Single-Family Home 001 (Master),Austin,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN


In [3]:
# turn egauge availability to integer then filter to anything > 90% available
meta_df.egauge_1min_data_availability = [re.sub('%','',str(x)) for x in meta_df.egauge_1min_data_availability.to_list()]
meta_df = meta_df.loc[(meta_df.active_record == 'yes')&(meta_df.egauge_1min_data_availability != 'nan')]
meta_df.egauge_1min_data_availability = meta_df.egauge_1min_data_availability.astype(int)
meta_df = meta_df.loc[(meta_df.egauge_1min_data_availability >= 90)]

# just for compute purposes
meta_df = meta_df.loc[(meta_df.city == 'Austin')|(meta_df.city == 'Ithaca')]

# remove nan time stamps
meta_df = meta_df.loc[(meta_df.egauge_1min_min_time != 'nan')&(meta_df.egauge_1min_max_time != 'nan')]

# get rid of tz then convert active times to datetime type
meta_df.egauge_1min_min_time = [re.sub(r'-0\d:00','',str(x)) for x in meta_df.egauge_1min_min_time]
meta_df.egauge_1min_max_time = [re.sub(r'-0\d:00','',str(x)) for x in meta_df.egauge_1min_max_time]
meta_df.egauge_1min_min_time = pd.to_datetime(meta_df.egauge_1min_min_time)
meta_df.egauge_1min_max_time = pd.to_datetime(meta_df.egauge_1min_max_time)

# analysis timeframe
meta_df = meta_df.loc[(meta_df.egauge_1min_min_time <= '2019-07-01')&(meta_df.egauge_1min_max_time >= '2020-08-01')]

# dtype to float & clean up na vals
meta_df['total_square_footage'] = meta_df.total_square_footage.astype(float)
meta_df.loc[(meta_df.city == 'Austin')&(meta_df.total_square_footage.isna()==True),'total_square_footage'] = np.median(meta_df.loc[(meta_df.city == 'Austin')].total_square_footage.dropna().to_list())
meta_df.loc[(meta_df.city == 'Ithaca')&(meta_df.total_square_footage.isna()==True),'total_square_footage'] = np.median(meta_df.loc[(meta_df.city == 'Ithaca')].total_square_footage.dropna().to_list())

# Ithaca is only single family homes, so Austin will be as well.
meta_df = meta_df.loc[(meta_df.building_type=='Single-Family Home 001 (Master)')]

# identify median size of Ithaca homes & find the absolute difference
ith_house_med = np.median(meta_df.loc[(meta_df.city == 'Ithaca')].total_square_footage.dropna().to_list())
meta_df['abs_diff_vec'] = [np.abs(ith_house_med - x) for x in meta_df['total_square_footage'].to_list()]

In [4]:
# ithaca pv proportion
ith_pv_prop = meta_df.loc[(meta_df.city == 'Ithaca'),
                          'pv'].value_counts()[0]/len(meta_df.loc[meta_df.city == 'Ithaca'])
# austin pv proportion
au_pv_prop = meta_df.loc[(meta_df.city == 'Austin'),
                         'pv'].value_counts()[0]/len(meta_df.loc[meta_df.city == 'Austin'])

# if ithaca pv proportion is lower than austin pv prop, 
# remove austin home with pv and highest absolute difference to ithaca
f_df = meta_df

while ith_pv_prop <= au_pv_prop:
    max_diff_idx = f_df.loc[(f_df.pv=='yes')&(f_df.city=='Austin'),'abs_diff_vec'].idxmax()
    f_df.drop(max_diff_idx,inplace=True)
    au_pv_prop = f_df.loc[(f_df.city=='Austin'),
                          'pv'].value_counts()[0]/len(f_df.loc[f_df.city=='Austin'])

ith_pv_prop = f_df.loc[(f_df.city=='Ithaca'),'pv'].value_counts()[0]/len(f_df.loc[f_df.city=='Ithaca'])
print(au_pv_prop,ith_pv_prop)

0.6516129032258065 0.6538461538461539


In [5]:
f_df.city.value_counts()

Austin    155
Ithaca     52
Name: city, dtype: int64

In [6]:
f_df.dataid.astype(int).to_list()

[545,
 3840,
 3383,
 10164,
 6526,
 3976,
 6069,
 1222,
 4090,
 3000,
 3700,
 2561,
 8278,
 6706,
 6672,
 153,
 3338,
 2096,
 6464,
 9186,
 387,
 6594,
 6514,
 5997,
 4550,
 4283,
 1417,
 7999,
 4628,
 6172,
 3488,
 2448,
 2811,
 7019,
 9004,
 6178,
 5679,
 5587,
 4414,
 950,
 558,
 5367,
 3517,
 9290,
 8627,
 690,
 5058,
 6487,
 6390,
 6161,
 8005,
 4580,
 3029,
 7690,
 4877,
 8450,
 5656,
 8198,
 621,
 8847,
 5450,
 186,
 8857,
 3310,
 994,
 668,
 503,
 8995,
 6348,
 796,
 6691,
 4633,
 2472,
 101,
 59,
 5317,
 5959,
 499,
 6558,
 781,
 3649,
 3506,
 6063,
 8645,
 9956,
 1086,
 1551,
 3893,
 2034,
 9484,
 3829,
 9141,
 3500,
 6121,
 9875,
 8292,
 946,
 5677,
 4356,
 3849,
 1947,
 3392,
 8386,
 7850,
 7739,
 7627,
 7108,
 5796,
 5246,
 5129,
 4526,
 4373,
 3723,
 3538,
 3134,
 2986,
 2980,
 2925,
 2470,
 1169,
 1103,
 252,
 187,
 9942,
 9939,
 9937,
 9932,
 9922,
 7989,
 7769,
 6730,
 6139,
 5949,
 5746,
 5218,
 3831,
 3778,
 2158,
 1796,
 1334,
 94,
 86,
 5439,
 8967,
 171,
 5892,
 7

In [7]:
f_df.loc[(f_df.city=='Austin')&(f_df.pv=='yes')&(f_df.egauge_1min_data_availability==100),['dataid','abs_diff_vec']].sort_values('abs_diff_vec')

,dataid,abs_diff_vec
1517,2814,35.0
1022,9937,46.0
433,3506,129.0
91,4090,198.0
352,8995,229.0
1311,5892,245.0
1571,8013,245.0
1327,5035,247.0
1118,7989,271.0
252,7690,291.0


In [25]:
pd.set_option('display.max_columns',999)
f_df.loc[(f_df.dataid == '10488')|(f_df.dataid == '5949'),:]

,dataid,active_record,building_type,city,state,egauge_1min_min_time,egauge_1min_max_time,egauge_1min_data_availability,egauge_1s_min_time,egauge_1s_max_time,egauge_1s_data_availability,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,bedroom2,bedroom3,bedroom4,bedroom5,battery1,car1,car2,circpump1,clotheswasher1,clotheswasher_dryg1,diningroom1,diningroom2,dishwasher1,disposal1,drye1,dryg1,freezer1,furnace1,furnace2,garage1,garage2,grid,heater1,heater2,heater3,housefan1,icemaker1,jacuzzi1,kitchen1,kitchen2,kitchenapp1,kitchenapp2,lights_plugs1,lights_plugs2,lights_plugs3,lights_plugs4,lights_plugs5,lights_plugs6,livingroom1,livingroom2,microwave1,office1,outsidelights_plugs1,outsidelights_plugs2,oven1,oven2,pool1,pool2,poollight1,poolpump1,pump1,range1,refrigerator1,refrigerator2,security1,sewerpump1,shed1,solar,solar2,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,winecooler1,wellpump1,water_ert_min_time,water_ert_max_time,water_data_availability,gas_ert_min_time,gas_ert_max_time,gas_data_availability,indoor_temp_min_time,indoor_temp_max_time,indoor_temp_data_availability,date_enrolled,date_withdrawn,house_construction_year,total_square_footage,first_floor_square_footage,second_floor_square_footage,third_floor_square_footage,half_floor_square_footage,lower_level_square_footage,pv,pv_panel_direction,total_amount_of_pv,amount_of_south_facing_pv,amount_of_west_facing_pv,amount_of_east_facing_pv,number_of_nests,audit_2011,audit_2013_2014,survey_2011,survey_2012,survey_2013,survey_2014,survey_2017,survey_2019,program_579,program_baseline,program_energy_internet_demo,program_lg_appliance,program_verizon,program_ccet_group,program_civita_group,program_shines,abs_diff_vec
1201,5949,yes,Single-Family Home 001 (Master),Austin,Texas,2012-10-31 19:00:00,2021-01-03 02:59:00,100,2019-03-11 19:00:00-05:00,2021-01-03 02:00:00-06:00,100%,yes,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,yes,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-09-01,NaN,1948.0,1962.0,1962.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,yes,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,CCET - Control,NaN,NaN,85.0
1555,10488,yes,Single-Family Home 001 (Master),Austin,Texas,2019-06-26 02:00:00,2021-01-03 02:59:00,100,2019-06-26 03:00:00-05:00,2021-01-03 02:00:00-06:00,100%,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-12,NaN,1942.0,1782.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,95.0


In [21]:
f_df

,dataid,active_record,building_type,city,state,egauge_1min_min_time,egauge_1min_max_time,egauge_1min_data_availability,egauge_1s_min_time,egauge_1s_max_time,egauge_1s_data_availability,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,bedroom2,bedroom3,bedroom4,bedroom5,battery1,car1,car2,circpump1,clotheswasher1,clotheswasher_dryg1,diningroom1,diningroom2,dishwasher1,disposal1,drye1,dryg1,freezer1,furnace1,furnace2,garage1,garage2,grid,heater1,heater2,heater3,housefan1,icemaker1,jacuzzi1,kitchen1,kitchen2,kitchenapp1,kitchenapp2,lights_plugs1,lights_plugs2,lights_plugs3,lights_plugs4,lights_plugs5,lights_plugs6,livingroom1,livingroom2,microwave1,office1,outsidelights_plugs1,outsidelights_plugs2,oven1,oven2,pool1,pool2,poollight1,poolpump1,pump1,range1,refrigerator1,refrigerator2,security1,sewerpump1,shed1,solar,solar2,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,winecooler1,wellpump1,water_ert_min_time,water_ert_max_time,water_data_availability,gas_ert_min_time,gas_ert_max_time,gas_data_availability,indoor_temp_min_time,indoor_temp_max_time,indoor_temp_data_availability,date_enrolled,date_withdrawn,house_construction_year,total_square_footage,first_floor_square_footage,second_floor_square_footage,third_floor_square_footage,half_floor_square_footage,lower_level_square_footage,pv,pv_panel_direction,total_amount_of_pv,amount_of_south_facing_pv,amount_of_west_facing_pv,amount_of_east_facing_pv,number_of_nests,audit_2011,audit_2013_2014,survey_2011,survey_2012,survey_2013,survey_2014,survey_2017,survey_2019,program_579,program_baseline,program_energy_internet_demo,program_lg_appliance,program_verizon,program_ccet_group,program_civita_group,program_shines,abs_diff_vec
25,545,yes,Single-Family Home 001 (Master),Austin,Texas,2012-08-12 19:00:00,2021-01-03 02:59:00,98,2018-02-14 13:03:58-06:00,2021-01-03 02:00:00-06:00,99%,yes,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,yes,NaN,NaN,NaN,yes,NaN,yes,yes,NaN,yes,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-04-12 18:33:48-05:00,2016-01-26 10:42:51-06:00,28%,NaN,NaN,NaN,2011-02-01,NaN,2007.0,2345.00,1385.0,960.0,0.0,NaN,NaN,yes,South;West,3.15,1.4,1.75,NaN,NaN,yes,yes,yes,NaN,yes,yes,NaN,yes,NaN,yes,yes,NaN,NaN,CCET - Text Message,NaN,NaN,468.00
48,3840,yes,Single-Family Home 001 (Master),Ithaca,New York,2019-05-22 19:00:00,2021-01-03 02:59:00,100,2019-05-22 19:00:00-05:00,2021-01-03 02:00:00-06:00,100%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,yes,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-06,NaN,1835.0,1484.00,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,393.00
70,3383,yes,Single-Family Home 001 (Master),Ithaca,New York,2019-05-21 19:00:00,2021-01-03 02:59:00,100,2019-05-21 19:00:00-05:00,2021-01-03 02:00:00-06:00,100%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN,NaN,NaN,yes,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-04,NaN,2002.0,1728.00,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.00
71,10164,yes,Single-Family Home 001 (Master),Ithaca,New York,2019-05-21 19:00:00,2021-01-03 02:59:00,100,2019-05-21 19:00:00-05:00,2021-01-03 02:00:00-06:00,100%,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,

In [9]:
# using PecanStreet_Owner account
#conn = pyodbc.connect(r'DRIVER={SQL Server};SERVER=ist-rs-sql1.ad.syr.edu, 1533;DATABASE=PecanStreet;UID=PecanStreet_Owner;PWD=#PecanStreet1;')

# select homes for 2020
#homes_2020 = pd.read_sql_query('SELECT * FROM [PecanStreet].[dbo].[p2020_exp_eng] WHERE dataid IN (545,3840,3383,10164,6526,3976,6069,1222,4090,3000,3700,2561,8278,6706,6672,153,3338,2096,6464,9186,387,6594,6514,5997,4550,4283,1417,7999,4628,6172,3488,2448,2811,7019,9004,6178,5679,5587,4414,950,558,5367,3517,9290,8627,690,5058,6487,6390,6161,8005,4580,3029,7690,4877,8450,5656,8198,621,8847,5450,186,8857,3310,994,668,503,8995,6348,796,6691,4633,2472,101,59,5317,5959,499,6558,781,3649,3506,6063,8645,9956,1086,1551,3893,2034,9484,3829,9141,3500,6121,9875,8292,946,5677,4356,3849,1947,3392,8386,7850,7739,7627,7108,5796,5246,5129,4526,4373,3723,3538,3134,2986,2980,2925,2470,1169,1103,252,187,9942,9939,9937,9932,9922,7989,7769,6730,6139,5949,5746,5218,3831,3778,2158,1796,1334,94,86,5439,8967,171,5892,7429,5615,2094,5035,4298,2378,2199,9729,9237,9160,9134,8829,8767,8236,8084,7731,6990,6836,5545,5403,4998,4336,4031,3967,3456,3009,2945,2859,2818,2814,2461,2335,1718,1642,1192,1185,744,7021,661,9002,11878,3403,4735,5192,984,10488,5763,9081,8013,370,9052,8156,7504,2129,7940,9333,1202,890,9121,7159,8162)',conn)

In [10]:
# using PecanStreet_Owner account
#conn = pyodbc.connect(r'DRIVER={SQL Server};SERVER=ist-rs-sql1.ad.syr.edu, 1533;DATABASE=PecanStreet;UID=PecanStreet_Owner;PWD=#PecanStreet1;')

# select homes for 2019
#homes_2019 = pd.read_sql_query("SELECT * FROM [PecanStreet].[dbo].[p2019_exp_eng] WHERE (dataid IN (545,3840,3383,10164,6526,3976,6069,1222,4090,3000,3700,2561,8278,6706,6672,153,3338,2096,6464,9186,387,6594,6514,5997,4550,4283,1417,7999,4628,6172,3488,2448,2811,7019,9004,6178,5679,5587,4414,950,558,5367,3517,9290,8627,690,5058,6487,6390,6161,8005,4580,3029,7690,4877,8450,5656,8198,621,8847,5450,186,8857,3310,994,668,503,8995,6348,796,6691,4633,2472,101,59,5317,5959,499,6558,781,3649,3506,6063,8645,9956,1086,1551,3893,2034,9484,3829,9141,3500,6121,9875,8292,946,5677,4356,3849,1947,3392,8386,7850,7739,7627,7108,5796,5246,5129,4526,4373,3723,3538,3134,2986,2980,2925,2470,1169,1103,252,187,9942,9939,9937,9932,9922,7989,7769,6730,6139,5949,5746,5218,3831,3778,2158,1796,1334,94,86,5439,8967,171,5892,7429,5615,2094,5035,4298,2378,2199,9729,9237,9160,9134,8829,8767,8236,8084,7731,6990,6836,5545,5403,4998,4336,4031,3967,3456,3009,2945,2859,2818,2814,2461,2335,1718,1642,1192,1185,744,7021,661,9002,11878,3403,4735,5192,984,10488,5763,9081,8013,370,9052,8156,7504,2129,7940,9333,1202,890,9121,7159,8162)) AND (localminute >= Convert(datetime, '2019-07-01' ))",conn)